# Second class

## Table of Contents (times are approximated)

1. [Summary of the first class (~5 min)](#summary)  
2. [Collaborative Filtering (~15 min)](#cf)  
   2.1 [Co-occurrence Matrix (~1.30h)](#copurchase)
   <br></br>
   2.2 [Memory-based CF (~1 h)](#memory-base)
   <br></br>
   2.3 [Model-based CF (~2 h)](#model-base)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os

## Summary of the first class in code <a id='summary'></a>

In [14]:
class movielens_100k(object):
    """
    This python class read the movielens-100k dataset and:
        . Preprocess train and test sets
        . Create a python dict that allows us to translate any item id to the corresponding movie title
    """
    
    def __init__(self, data_root, item_id_file='u.item', train_file='ua.base', test_file='ua.test', 
                 columns=['user_id', 'item_id', 'rating', 'timestamp']):
                
        self.data_root = data_root
        self.items_id_file = os.path.join(self.data_root, item_id_file)
        self.train_file = train_file
        self.test_file = test_file
        self.columns = columns
        self.__data = {}  # dictionary to store train and test data
            
    def __getitem__(self, dataset):
        """
        Get train and test data

        :param dataset: dataset name (String)
        :return: pandas dataframe
        """
        return self.__data[dataset]

    def __setitem__(self, key, value):
        """
        Store train and test data in the dictionary self.__data.

        :param key: name of the dataset (String)
        :param value: pandas dataframe
        """
        self.__data[key] = value
        
    def read_data(self, file, verbose=False):
        """
        Read file returning a pandas dataframe
        :param file: file path
        :param verbose: print the first rows of the data
        """
        datafile = os.path.join(self.data_root, file)
        data = pd.read_csv(datafile, sep='\t', names=self.columns)
        if verbose:
            print(data.head())
        return data
            
    def returnItemId(self, text, dict_ids):
        """
        Retrieves all the ids and titles for movies containing 'text' in its title
        :param text: string to be looked for in movies titles
        :param dict_ids: dicttionary of {id:title}

        :return: a list of (id,title) if text found in titles, and an empty list otherwise.
        """
        # convert input text to lowercase
        text_ = text.lower()
        # find occurances
        search = [(k, v.lower().find(text_)) 
                  for k,v in list(dict_ids.items())]

        # Get the IDs corresponding to the given text
        index = [k for k,v in search if v>-1]

        # Return a list with the id and the name
        out = []
        for i in index:
            out.append((i, dict_ids[i]))
        return out
    
    def check_data_consistency(self):
        """
        Check if one film has two or more ids
        """
        self.item_dict = {}
        with io.open(self.items_id_file, 'rb') as f:
            for line in f.readlines():
                record = line.split(b'|')
                self.item_dict[int(record[0])] = str(record[1])
                
        if len(set(self.item_dict.keys())) == len(set(self.item_dict.values())):
            self.unique_item_dict_ = self.item_dict
            return True
        else:
            self.duplicates_item_dict = {}
            for id_,name in list(self.item_dict.items()):
                if name not in self.duplicates_item_dict:
                    self.duplicates_item_dict[name] = [id_]
                else:
                    self.duplicates_item_dict[name] = self.duplicates_item_dict[name]+[id_]

            self.unique_id_item_dict ={}
            for i, lista_id in enumerate(self.duplicates_item_dict.values()) :
                for key in lista_id:
                    self.unique_id_item_dict[key] = i

            self.unique_item_dict_ = {self.unique_id_item_dict[k]:v 
                                for k,v in self.item_dict.items()}
            return False
    
    def correcting_non_unique_movies(self, data_df):
        """
        Preprocess data_df in case the check_data_consistency function fails
        """
        if not self.check_data_consistency():
            data_df['item_id'] = data_df['item_id'].apply(lambda x: self.unique_id_item_dict[x])
            
        return data_df
            
    def train(self, verbose=False):
        """
        Train data
        """
        self['train_set'] = self.read_data(self.train_file, verbose)
        self['train_set'] = self.correcting_non_unique_movies(self['train_set'])
        items = self['train_set'].item_id.unique()
        self.train_id_dict = {j:i for i,j in enumerate(items)}
        self['train_set']['item_id'] = self['train_set']['item_id'].apply(lambda x: self.train_id_dict[x])
        self.unique_item_dict = {self.train_id_dict[v]:self.unique_item_dict_[v] 
                                for v in items}
        self.n_users = self['train_set'].user_id.unique().shape[0]
        self.n_items = self['train_set'].item_id.unique().shape[0]
        self.n_pairs = self['train_set'].shape[0]
        if verbose:
            print('There are %s users, %s items and %s pairs in the test set' \
                  %(self.n_users, self.n_items, self.n_pairs))
    
    def test(self, verbose=False):
        """
        Test data
        """
        self['test_set'] = self.read_data(self.test_file, verbose)
        self['test_set']['item_id'] = self['test_set']['item_id'].apply(lambda x: self.train_id_dict[self.unique_id_item_dict[x]] if self.unique_id_item_dict[x] in self.train_id_dict else -1)
        self['test_set'] = self['test_set'][self['test_set']['item_id']!=-1]
        if verbose:
            print('There are %s users, %s itmes and %s pairs in the test set' \
                  %(self['test_set'].user_id.unique().shape[0], self['test_set'].item_id.unique().shape[0], 
                    self['test_set'].shape[0]))
            
    def get_data(self, verbose=False):
        """
        A kind of main function to return train and test data together
        """
        self.train(verbose)
        self.test(verbose)

In [15]:
# Examples:
movielens = movielens_100k("ml-100k/")
movielens.get_data() #Get train and test data

In [16]:
movielens['train_set'].head()

,user_id,item_id,rating,timestamp
0,1,0,5,874965758
1,1,1,3,876893171
2,1,2,4,878542960
3,1,3,3,876893119
4,1,4,3,889751712


In [17]:
movielens['test_set'].head()

,user_id,item_id,rating,timestamp
0,1,906,4,887431883
1,1,659,4,878542699
2,1,725,4,878542420
3,1,425,3,874965739
4,1,660,2,878542201


In [18]:
print(list(movielens.unique_item_dict[item_id] for item_id in [1,5,20]))

["b'GoldenEye (1995)'", "b'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'", "b'Braveheart (1995)'"]


In [19]:
class popularity_models(object):
    """
    Implement the following popularity models:
        . Most rated movie (it is assumed that this is the most watched movie)
        . Most positively rated movie (rating > a given threshold)
        . Highest rated movie
    """
    def __init__(self, model, train_df):
        self.model_name = model
        self.train_df = train_df
    
    def mostRated(self):
        """
        Most rated movie model
        """
        return self.train_df.groupby('item_id')['user_id'].count().sort_values(ascending=False)

    def positiveRated(self, min_rating=4.0):
        """
        Most positively rated movie model (rating > min_rating)
        """
        return self.train_df[self.train_df.rating>=min_rating].groupby('item_id')['user_id'].count().sort_values(ascending=False) 
        
    def mean_rate_movies(self, min_ratings=50):
        """
        Highest rated movie model. Only items with more than min_ratings ratings are considered
        """
        listRatedMovies = self.train_df.groupby('item_id')['rating'].apply(list).reset_index()
        filteredListRatedMovies = listRatedMovies[listRatedMovies.rating.apply(lambda x: len(x)>min_ratings)]
        meanMovies = filteredListRatedMovies.rating.apply(lambda x: np.mean(np.array(x))).sort_values(ascending=False)
        return meanMovies
        
    def train(self, min_rating=4.0, min_ratings=50):
        """
        Train a popularity model (i.e., sort the item list according to a particular frequency metric)
        """
        if self.model_name == 'mostRated':
            self.model = self.mostRated()
        elif self.model_name == 'positiveRated':
            self.model = self.positiveRated(min_rating)
        elif self.model_name == 'mean_rate_movies':
            self.model = self.mean_rate_movies(min_ratings)
        else:
            raise ValueError('%s doesnot exist' % self.model_name)
        
    def predict(self, top_k, verbose=False):
        """
        Return the first top_k recommendation of the chosen popularity model
        """
        return self.model.index.values[:top_k]
    
    def get_titles(self, movies_ids, unique_item_dict):
        """
        Given a list of movies ids, this method return their titles
        """
        title = []
        for i, index in enumerate(movies_ids):
            id_ = index # id of the movie
            title.append(unique_item_dict[id_])

        print(title)
        

In [20]:
# Example: Mean rate movie model
p = popularity_models('mean_rate_movies', movielens['train_set'])
p.train()
p.model.head()

110    4.491525
378    4.480769
162    4.476636
299    4.475836
452    4.459821
Name: rating, dtype: float64

In [21]:
p.predict(3)

array([110, 378, 162])

In [22]:
p.get_titles(p.predict(3), movielens.unique_item_dict)

["b'Wallace & Gromit: The Best of Aardman Animation (1996)'", "b'Close Shave, A (1995)'", "b'Wrong Trousers, The (1993)'"]


In [23]:
class Metrics():
    """
    Implement the two most important relevance's metric for RS:
        . Recall
        . MAP
    """
    def __init__(self, train_ids, test_ids, recommended_ids):
        
        self.zipped = list(zip(test_ids, train_ids, recommended_ids))
        
    def recall_per_user(self, N, test, recommended, train):
        """
        :param N: number of recommendations
        :param test: list of movies seen by user in test
        :param recommended: list of movies recommended

        :return the recall
        """   
        if train is not None: 
            rec_true = []
            for r in recommended:
                if r not in train:
                    rec_true.append(r)
            else:
                rec_true = recommended    
        intersection = len(set(test) & set(rec_true[:N]))
        return intersection / float(np.minimum(N, len(test)))
       
    def recall_at_n(self, topN):
        """
        Computes the mean recall@k, for k in topN
        :param topN: list of positions
        """
        out = []
        for k in topN:
            recall = np.mean([self.recall_per_user(k, test, recom, train)  for (test, train, recom) in self.zipped])
            out.append(recall)
            print("recall@%s=%.3f" %(k, recall))
        return out
                                       
    def apk(self, N, test, recommended, train):
        """
        Computes the average precision at N given recommendations.

        :param N: number of recommendations
        :param test: list of movies seen by user in test
        :param recommended: list of movies recommended

        :return The average precision at N over the test set
        """   
        if train is not None: 
            rec_true = []
            for r in recommended:
                if r not in train:
                    rec_true.append(r)
        else:
            rec_true = recommended    
        predicted = rec_true[:N] # top-k predictions

        score = 0.0 # This will store the numerator
        num_hits = 0.0 # This will store the sum of rel(i)

        for i,p in enumerate(predicted):
            if p in test and p not in predicted[:i]:
                num_hits += 1.0
                score += num_hits/(i+1.0)

        return score / min(len(test), N)

    def map_at_n(self, topN):
        """
        Computes the mean average precision at k, for k in topN
        :param topN: list of positions
        """
        out = []
        for k in topN:
            map_ = np.mean([self.apk(k, test, recom, train)  for (test, train, recom) in self.zipped])
            out.append(map_)
            print("map@%s=%.3f" %(k, map_))
        return out

In [24]:
# Example
topk = [5, 10, 20, 50]
trainUsersGrouped = movielens['train_set'].groupby('user_id')['item_id'].apply(list).reset_index().sort_index()
train_ids = trainUsersGrouped.item_id.values
testUsersGrouped = movielens['test_set'].groupby('user_id')['item_id'].apply(list).reset_index().sort_index()
test_ids = testUsersGrouped.item_id.values
predictions = trainUsersGrouped.item_id.apply(lambda x: p.predict(movielens.n_items)).reset_index()
predictions_ids = predictions.item_id.values
m = Metrics(train_ids, test_ids, predictions_ids)
print('*****Recall*****')
_ = m.recall_at_n(topk)
print('*****Map*****')
_ = m.map_at_n(topk)


*****Recall*****
recall@5=0.016
recall@10=0.024
recall@20=0.048
recall@50=0.112
*****Map*****
map@5=0.011
map@10=0.009
map@20=0.013
map@50=0.018


<img src="https://courses.edx.org/c4x/BerkeleyX/CS100.1x/asset/Collaborative_filtering.gif" alt="collaborative filtering" style="float: right; width: 300px"/>

## 4. Collaborative Filtering <a id='cf'></a>

Perhaps, one of the most succesful techniques for making personalized recommendations are the so called *collaborative filtering* (CF) algorithms. CF is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue X than to have the opinion on X of a person chosen randomly. 

The image at the right (from Wikipedia) shows an example of user's preference prediction using collaborative filtering. At first, people rate different items (like videos, images, games). After that, the system is making predictions about a user's rating for an item, which the user has not rated yet. These predictions are built upon the existing ratings of other users, who have similar ratings with the active user. For instance, in the image at the right the system has made a prediction, that the active user will not like the video.

In this part we will see three kinds of CF, of increasing complexity:

4.1 [CF with co-occurrence](#copurchase)

4.2 [Memory-based CF](#memory-base)

4.3 [Model-based CF](#model-base)

<a id='copurchase'></a>
## 4.1 Co-occurrence Matrix

The idea is to recommend movies similar to the movies already seen by a user. A measurement of similarity among items is obtained from the co-occurrence matrix. This is nothing else than the adjacency matrix of the graph of items created by users!!!

<table border="0" style="width:825px;border:0px;">
<tr>
    <td> 
        <img src="https://lucidworks.com/wp-content/uploads/2015/08/Les-Miserables-Co-Occurrence.png" style="width: 500px"/>
    </td>
    <td> 
        <img src="https://lucidworks.com/wp-content/uploads/2015/08/midnight-club-graph.png" style="width: 400px"/>
    </td>
</tr>
</table>


Define the co-ocurrance matrix, which have shape=[n_items,n_items]. Filter movies rated with rating >=4.0

Suggested steps:
* create a dictionary of movies per user: moviesPerUser ={user: array[movies]}
* co-ocurrance matrix: coMatrix

Visualize the matrix with the following piece of code:



In [ ]:
plt.matshow(coMatrix, fignum=1000, cmap=plt.cm.binary)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

<div class="alert alert-success">
**QUESTION:** Can you think of a better way of visualizaing this matrix? Try to rescale it, or to rearrenge it follwoing some criteria (for instance, popularity!).
</div>

#### 4.1.1 Making predictions using the co-occurrence matrix

This kind of recommendations, based on item similarity, provide a measure of the closeness of one item to another. In order to make a recommendation for a user, we have to proceed as follows:

* First, define a function that returns the top-N closest items to a given one.
* Then, for a list of items adopted by a specific user, select the top-N items from the lists of top-N closest items to each adopted item.

In [82]:
#TODO
def co_occurrance_similarity(item_id, coocurrance, ntop=10):
    """
    Returns the top-N most similar items to a given one, based on the coocurrance matrix
    
    :param item_id: id of input item
    :param cooccurrance: 2-dim numpy array with the co-occurance matrix
    :param ntop: number of items to be retrieved
    
    :return top-N most similar items to the given item_id
    """

    return 

In [ ]:
#TODO. Run the co_occurrance_similarity method



In [ ]:
#TODO: Play with the movie ID, and print out its first Ntop recommendations!!



Now, let use this function to make recommendations:

In [86]:
#TODO
def co_occurrance_recommendation(items_id, cooccurrance, ntop=10):
    """
    Obtain the list of ntop recommendations based on a list of items (user history of views)
    
    :param items_id: list of items ids
    :param coocurrence: co-ocurrence matrix (numpy 2-dim array)
    :param ntop: top-K items to be retrieved
    
    :return list of ntop items recommended
    """
    
    return 

Get the predictions

Note that, unlike previous popularity based models, the recommendation is now (slightly) different from one user to another

#### Evalute the recommendation

Get the `recall_at_n` for n = [3,10,30,50,100]

*Do the same analysis for the map metric*

<div class = "alert alert-info">
Compare this results to those obtained with the popularity model. Was it so bad?
</div>

Some comments:
* The dataset we are using here is very simple. Indeed, you can see that the popularity baseline achieves quite decent metric values. This won't happen in a real world dataset! The reason it happens here is because the dataset is quite small, and quite biased towards popular items. 

* Recall does not account for the order of the recommendation, while map does. This explains why the co-occurrance model performs better after the first 10 recommendations in terms of map (ordering), while recall values are always better for the popularity based model.  


### 4.1.2 Oher distances

So far, we have defined the *closeness* of two items as the number of users shared. However, it would make make sense to define it relative the total number of users that have watch a movie. This can be done with the [Jaccard similarity index](https://en.wikipedia.org/wiki/Jaccard_index):

$$J(i,j)=\frac{|i\cap j|}{|i|+|j|-|i\cap j|}\in [0,1]$$


<div class = "alert alert-success">
Build the Jaccard similarity matrix from the co-occurrance matrix. Notice that $CoM(i,j) = |i\cap j|$ and $CoM(k,k) = |k|$. In addition, if both $|i|=0$ and $|j|=0$, the similarity is defined as 1 (this is a convention).
</div>

Define the Jaccard similarity matrix: jaccard

As before, let's visualize the matrix:

In [ ]:
plt.matshow(jaccard, fignum=1000, cmap=plt.cm.binary)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

Rearrange following popularity criteria (avoid ones!):


Recommend using Jaccard similarity:

#### Evaluate the recommendations

Recall@[3,10,30,50,100]

Map@[3,10,30,50,100]

<a id='memory-base'></a>
## 4.2. Memory-Based Collaborative Filtering (CF)

Although the methods developed so far return a list of recommended items, they cannot be used to make an actual prediction regarding the rating. A quite different approach would be to calculate the unknown rating, $r_{ui}$, as the averaged of some other ratings, thta are somehow close to either the user or the item in question. 

Thus, one approach is to take

### $$r_{u,i} = \frac{1}{K}\sum_{j\in\mathcal{I}'} \mathrm{sim}(i,j) r_{u,j},$$

where items $j\in\mathcal{I}'$ are taken from the set of $K$ closest items to $i$, or from the whole dataset. This is known as **item-item collaborative filtering**, and can be interpreted as *“users who liked this movie also liked …”*. See Amazon famous patent: https://www.google.com/patents/US7113917. Basically, this technique will take an item, find users who liked that item, and find other items that those users or similar users also liked. 

Similarly, one can define a **user-user filtering** where predictions are made as

### $$r_{u,i} = \frac{1}{K} \sum_{v\in\mathcal{U}'} \mathrm{sim}(u,v) r_{v,i}.$$

<img src="https://soundsuggest.files.wordpress.com/2013/06/utility_matrix.png" alt="utility matrix" style="float: right; width: 400px"/>

In this case, the recommendation would be more like *“users who are similar to you also liked …”*. Both techniques are part of the broad familiy of **Memory-Based Collaborative Filtering** approaches, or neighborhood-based algorithms.

The similarity among users or items can be calculated in a variety of forms: Pearson's correlation, cosine distance, etc. Here we will use the cosine distance. For this, we will first create the utility user-item matrix. 

The utility matrix is a dense representation of the user-item intearction. We have been using the *long* format, where missing entries are obviated; now, we will use the *wide* format, i.e. the matrix representation (see the figure on the right). 

<br></br>
<div class = "alert alert-info">
** NOTE **: Long and wide formats have its benefits and drawbacks. Can you think of some of them?
</div>

Put the train and test datasets in wide format (i.e., like a matrix). 

Name this variables uMatrixTraining and uMatrixTesting. 

Important!! Note user ids start at 1!

#### Define a similarity measure: cosine similarity

#### $$\mathrm{sim}({\bf a},{\bf b})=\frac{{\bf a}\cdot{\bf b}}{\sqrt{{\bf a}\cdot{\bf a}}\sqrt{{\bf  b}\cdot{\bf b}}}$$

In [117]:
#TODO
def cosineSimilarity(ratings, kind='user', epsilon=1e-9):
    """
    Calculate the cosine distance along the row (columns) of a matrix for users (items)
    
    :param ratings: a n_user X n_items matrix
    :param kind: string indicating whether we are in mode 'user' or 'item'
    :param epsilon: a small value to avoid dividing by zero (optional, defaults to 1e-9)
    
    :return a square matrix with the similarities
    """
    # epsilon -> small number for handling dived-by-zero errors
    return 

### 4.2.1. User-user CF

*“Users who are similar to you also liked …”*

### $$r_{u,i} = \frac{1}{K} \sum_{v\in\mathcal{U}'} \mathrm{sim}(u,v) r_{v,i}.$$
Consider user $x$:

1. Find other users whose ratings are “similar” to $x$’s ratings, i.e. calculate the similarity among users with the help of the cosineSimilarity function.
2. Estimate missing ratings based on ratings of similar users

Have a look at the userSimilarity matrix:

In [119]:
cax = plt.matshow(userSimilarity, fignum=1000, cmap=plt.cm.coolwarm)
plt.gcf().colorbar(cax, ticks=[0, 0.25, 0.5])
plt.clim(0, 0.5)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

Now, get userItemCFpredictions

*Note that if we multiply `userSimilarity` by `uMatrixTraining` we get the ratings weigthed with user similar similarity. Then, we have to normalize by the average similarity for each user*

*Note that some users might give generally lower ratings than others, so that we could have also corrected for this effect as follows*

In [126]:
sum_ = uMatrixTraining.sum(axis=1)
len_ =np.count_nonzero(uMatrixTraining, axis=1)
average_ratings = np.tile(sum_/ len_, n_items).reshape([n_items, n_users]).T
uMatrixTraining_shifted = uMatrixTraining - np.multiply(average_ratings, uMatrixTraining)
userItemCFpredictions_corrected = average_ratings + userSimilarity.dot(uMatrixTraining_shifted) / norm

### 4.2.2. Item-Item CF

*“Users who liked this movie also liked …”*

Consider item $i$, and following the previous section:

1. For item $i$, find other similar items, i.e. calculate the similarity itemSimilarity among items as before we did for users
2. Estimate rating for item $i$ based on ratings for similar items: itemItemCFpredictions



### 4.2.3 Show some recommendations

In case of item-item CF, the recommendation is pretty much the same as with the co-occurence matrix. It's also quite simple to find similar items to a given one.

Find movies similar to a given one using the item-item similarity matrix.



Calculate the recommendations obtained with the item-item CF model.


<div class="alert alert-success">
Do the same with the user-user CF model.
</div>

**As you can see, the recommendations are pretty bad... Let's measure that**

### 4.2.4 Measure the recommendations

Since we are predicting ratings, it might make sense to introduce a metric that accounts for this. In particular, the **Root Mean Square Error (RMSE)** is typically used for this purpose. 

### $$\mathrm{RMSE}=\sqrt{\frac{1}{n_{\mathrm{users}}n_{\mathrm{items}}}\sum_{u,i}\left(r_{u,i}-\hat{r}_{u,i}\right)^2}$$

In [139]:
#TODO
from math import sqrt
def rmse(prediction, ground_truth):
    """
    Return the Root Mean Squared Error of the prediction
    
    :param prediction: a 2-dim numpy array with the predictions
    :param ground_truth: a 2-dim numpy array with the known ratings
    
    :return the RMSE
    """

    return 

### Calculate ranking metrics

Use recall_at_n, with n i in [10, 100]

<a id='model-base'></a>
## 4.3. Model-based CF or Latent factor models
There are several model-based CF: from matrix factorizations to bayesian models, neural netwroks, etc. In all of them, we try to extract latent factors (vectors) that model user and item interactions. In contrast to previous methods, our hypothesis here is that the dimension of the latent spaces is rather small (in the order of a hundred dimensions). Then, we use this latent features to make a prediction:

## $$r_{u,i} \approx {\bf f}_u^T\cdot{\bf f}_i$$

The underlying assumption is that both users and items *live* in the same latent space, and that we can unravel such space. 

<img src="https://www.researchgate.net/profile/Tunca_Dogan/publication/235913413/figure/fig3/AS:299678856957952@1448460415040/Figure-3-The-distribution-of-the-points-in-the-Swiss-roll-dataset-in-3-D-space.png
" alt="swiss roll" style="float: center; width: 300px"/>


Here we will use a couple of linear Matrix Factorization (MF) models:

* Singular Value decomposition (SVD)
* Alternating Least Squares (ALS)

### 4.3.1 Singular value decomposition

The main idea is to reduce the dimensionality of the input space. This is pretty much the same as Eigen-decomposition or Principal Component Analysis (PCA)-

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/GaussianScatterPCA.svg/220px-GaussianScatterPCA.svg.png
" alt="dimensionaly reducion" style="float: center; width: 500px"/>


In [148]:
from scipy.sparse.linalg import svds

Get SVD components from train matrix. Choose k=20.


Take a look at the different matrices
* check the shapes
* check if U and V are orthogonal matrices with the left singular vectors as columns

### Get the recommendations

We will reconstruct the utility matrix R (i.e., the recommendation matrix) as follows:

### $$M\approx U\mathrm{diag}(s)V^T$$

First, build a diagonal matrix s_diag_matrix with the eigenvalues




Make the prediction: svdPredictions

Check if the dimensions of svdPredictions and uMatrixTesting are correct

### Evaluate the model

* Recall@{10,100}
* Play with more dimensions: svd_dim in {10, 50, 100}

### Implicit vs Explicit feedback

In the above SVD matrix factorization we have tried to reconstructt the matrix of ratings. However, it might be easier trying to model the matrix of preferences (i.e., wether the user likes or not a movie).

For this, we will define a “selector” matrix $I$ for the training utility matrix $R$, which will contain 0 if the rating matrix has no rating entry, and 1 if the rating matrix contains an entry. And the smae for testing data.


Convert uMatrixTraining and uMatrixTesting into implicit data (I and I2 respectively). For these new matrices, if r>3 then 1, else = 0

Evaluate SVD with RMSE, and recall@k and map@k, with k in [10, 30, 50, 100]

### Overfitting

As we introduce more dimensions in the model, we make it more prone to overfit. This can be observed in the decrease of error (RMSE) in train, while it increases in test.

In [ ]:
error_train = []
error_test = []
dims = [1, 2, 5, 7, 10, 20, 40, 80, 120, 160, 200]
for dim in dims:
    print("*"*30)
    print("Using %s latent dimensions" %dim)
    # apply Singular Value Decomposition
    u, s, vt = svds(I, dim)
    s_diag_matrix = np.diag(s)
    # make the prediction
    svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
    error_train.append(rmse(svdPredictions, I))
    error_test.append(rmse(svdPredictions, I2)) 
plt.semilogx(dims, error_train, '--*b', label="train")
plt.semilogx(dims, error_test, '--*g', label="test")
plt.xlabel("Numer of latent dimensions")
plt.ylabel("Error")
plt.legend(loc=7)
plt.show()

**Note that after few dimensions (~10) the model starts to overfit (the error in test increases). Thus, we need to use other method to regularize the model (i.e., set restrictions/limitations to the model, so that it cannot overfit)**

### 3.2 Alternating Least Squares (ALS)

SVD can be very slow and computationally expensive. Besides, when addressing only the relatively few known entries we are highly prone to overfitting.

An scalable alternative to SVD is ALS, which can include regularization terms to prevent overfitting. We will rename our variable to make them more similar to the ALS notation

### ALS algorithm

The ALS algorithm aims to estimate two unknown matrices which, when multiplied together, yield the rating matrix. The loss function you will use is the well-known sum of squared errors. The second term is for regularisation to prevent overfitting

<img src="https://latex.codecogs.com/gif.latex?\underset{Q*&space;,&space;P*}{min}\sum_{(u,i)\epsilon&space;K&space;}(r_{ui}-P_u^TQ_i)^2&plus;\lambda(\left&space;\|&space;Q_i&space;\right&space;\|^2&space;&plus;&space;\left&space;\|&space;P_u&space;\right&space;\|^2)$&space;&space;$" title="\underset{q* , p*}{min}\sum_{(u,i)\epsilon K }(r_{ui}-q_i^Tp_u)^2+\lambda(\left \| q_i \right \|^2 + \left \| p_u \right \|^2)" />

The Alternating Least Squares algorithm does this by first randomly filling the users matrix with values and then optimizing the value of the movies such that the error is minimized.  Then, it holds the movies matrix constant and optimizes the value of the user's matrix.  This alternation between which matrix to optimize is the reason for the "alternating" in the name. 

<img alt="factorization" src="http://spark-mooc.github.io/web-assets/images/matrix_factorization.png" style="width: 885px"/>
<br clear="all"/>

This optimization is what's being shown on the right in the image above.  Given a fixed set of user factors (i.e., values in the users matrix), we use the known ratings to find the best values for the movie factors using the optimization written at the bottom of the figure.  Then we "alternate" and pick the best user factors given fixed movie factors.

It must be noticed that this is another way of reducing the dimensionality of the input matrix (like PCA, or more generally, SVD). This has important consequences:

* ### Our decomposition is linear. We won't be able to catch non-linear relationships among users and items.
* ### As in PCA or SVD, our features will correspond to directions of maximum variance in the data. Thus, the first feature will catch most of this variation, the second, a little bit more, and so on. It implies that the error in the reconstruction will not decrease dramatically when using more features!!! Keep this in mind.


In [14]:
def alsRmse(I,R,Q,P):
    return 

Train ALS. Check the convergence using check_als_performance function

In [ ]:
# Check performance by plotting train and test errors
import matplotlib.pyplot as plt
%matplotlib inline

def check_als_performance(train_errors, test_errors):
    plt.plot(range(n_epochs), train_errors, marker='o', label='Training Data');
    plt.plot(range(n_epochs), test_errors, marker='v', label='Test Data');
    plt.title('ALS-WR Learning Curve')
    plt.xlabel('Number of Epochs');
    plt.ylabel('RMSE');
    plt.legend()
    plt.grid()
    plt.show()

### ALS evaluation
* RMSE
* recall@[10, 30, 50, 100]
* map@[10, 30, 50, 100]